# 03 - Consolidação de Jogadores

Este notebook realiza:
1. Criação de chave única (player_id + competition_id + team_id)
2. Marcação de registro atual (v_current = True para mais recente)
3. Geração de colunas auxiliares

**Nota**: Mantém todos os registros históricos. Use `v_current == True` para filtrar apenas os mais recentes.

In [ ]:
import pandas as pd
from pathlib import Path

BASE_DIR = Path("c:/jobs/botafogo/v3")
OUTPUT_DIR = BASE_DIR / "bases" / "outputs"

## 1. Carregar Dados

In [ ]:
# Carregar dados com posições mapeadas
df = pd.read_parquet(OUTPUT_DIR / "_temp_scouts_positions.parquet")
print(f"Dados carregados: {len(df)} registros")

In [ ]:
# Verificar colunas de identificação
print("Colunas de identificação disponíveis:")
id_cols = ["player_id", "competition_id", "team_id", "player_season_most_recent_match"]
for col in id_cols:
    if col in df.columns:
        print(f"  {col}: {df[col].nunique()} valores únicos")

## 2. Criar Chave Única (player_id + competition_id + team_id)

In [ ]:
# Criar chave única: player_id + competition_id + team_id
df["unique_key"] = (
    df["player_id"].astype(str) + "_" +
    df["competition_id"].astype(str) + "_" +
    df["team_id"].astype(str)
)

print(f"Total de registros: {len(df)}")
print(f"Chaves únicas: {df['unique_key'].nunique()}")
print(f"Registros duplicados: {len(df) - df['unique_key'].nunique()}")

In [ ]:
# Verificar duplicatas
duplicates = df[df.duplicated(subset="unique_key", keep=False)]
if len(duplicates) > 0:
    print(f"Registros com chave duplicada: {len(duplicates)}")
    print("\nExemplo de duplicata:")
    sample_key = duplicates["unique_key"].iloc[0]
    display(duplicates[duplicates["unique_key"] == sample_key][["player_id", "competition_id", "team_id", "player_season_most_recent_match", "source_file"]])
else:
    print("Nenhuma duplicata encontrada!")

## 3. Marcar Registro Atual (v_current)

In [ ]:
# Criar coluna v_current para identificar o registro mais recente por unique_key
# Mantém TODOS os registros, apenas marca qual é o atual

if "player_season_most_recent_match" in df.columns:
    # Converter para datetime se necessário
    df["player_season_most_recent_match"] = pd.to_datetime(df["player_season_most_recent_match"], errors="coerce")
    
    # Ordenar por data (mais recente primeiro)
    df = df.sort_values("player_season_most_recent_match", ascending=False)
    
    # Marcar o primeiro de cada grupo (mais recente) como v_current = True
    df["v_current"] = ~df.duplicated(subset="unique_key", keep="first")
else:
    # Se não tiver a coluna de data, marcar o primeiro encontrado como atual
    df["v_current"] = ~df.duplicated(subset="unique_key", keep="first")

print(f"Total de registros: {len(df)}")
print(f"Registros atuais (v_current=True): {df['v_current'].sum()}")
print(f"Registros históricos (v_current=False): {(~df['v_current']).sum()}")

In [ ]:
# Verificar distribuição
print("\nDistribuição de v_current:")
print(df["v_current"].value_counts())

## 4. Gerar Colunas Auxiliares

In [ ]:
# Verificar colunas de nome
name_cols = [c for c in df.columns if "name" in c.lower()]
print("Colunas com 'name':")
for col in name_cols:
    print(f"  {col}")

In [ ]:
# Criar coluna auxiliar player_name com lógica de prioridade:
# 1. player_known_name (prioridade máxima)
# 2. player_name (vem do Excel original)
# 3. first_name + last_name (somente se AMBOS existirem)
# 4. somente first_name
# 5. somente last_name
# 6. Se nenhum nome disponível, deixar como None

# Criar uma nova coluna temporária para o nome final
df["player_name_final"] = None

# Prioridade 1: player_known_name
if "player_known_name" in df.columns:
    mask = df["player_known_name"].notna()
    df.loc[mask, "player_name_final"] = df.loc[mask, "player_known_name"]

# Prioridade 2: player_name (do Excel original)
if "player_name" in df.columns:
    mask = df["player_name_final"].isna() & df["player_name"].notna()
    df.loc[mask, "player_name_final"] = df.loc[mask, "player_name"]

# Prioridade 3: first_name + last_name (somente se AMBOS existirem)
if "player_first_name" in df.columns and "player_last_name" in df.columns:
    mask = (
        df["player_name_final"].isna() & 
        df["player_first_name"].notna() & 
        df["player_last_name"].notna()
    )
    df.loc[mask, "player_name_final"] = (
        df.loc[mask, "player_first_name"].astype(str) + " " + df.loc[mask, "player_last_name"].astype(str)
    )

# Prioridade 4: somente first_name
if "player_first_name" in df.columns:
    mask = df["player_name_final"].isna() & df["player_first_name"].notna()
    df.loc[mask, "player_name_final"] = df.loc[mask, "player_first_name"]

# Prioridade 5: somente last_name
if "player_last_name" in df.columns:
    mask = df["player_name_final"].isna() & df["player_last_name"].notna()
    df.loc[mask, "player_name_final"] = df.loc[mask, "player_last_name"]

# Substituir a coluna player_name pela versão final
df["player_name"] = df["player_name_final"]
df.drop(columns=["player_name_final"], inplace=True)

# Mostrar estatísticas de nomes
print(f"Total de registros: {len(df)}")
print(f"Registros com player_name: {df['player_name'].notna().sum()}")
print(f"Registros SEM player_name: {df['player_name'].isna().sum()}")

# competition_name - nome da competição
if "competition_name" not in df.columns and "competition_id" in df.columns:
    # Usar source_file como proxy
    df["competition_name"] = df["source_file"].str.replace(".xlsx", "", regex=False)

# team_name - nome do time
if "team_name" not in df.columns:
    team_cols = [c for c in df.columns if "team" in c.lower() and "name" in c.lower()]
    if team_cols:
        df["team_name"] = df[team_cols[0]]

print("Colunas auxiliares criadas!")

In [ ]:
# Verificar resultado
display_cols = ["player_id", "player_name", "competition_id", "team_name", "mapped_position", "v_current"]
available_cols = [c for c in display_cols if c in df.columns]

print("Amostra dos dados (todos os registros):")
df[available_cols].head(10)

In [ ]:
# Exemplo: filtrar apenas registros atuais
df_current = df[df["v_current"] == True]
print(f"\nAmostra filtrada (v_current=True):")
print(f"Total: {len(df_current)} registros")
df_current[available_cols].head(10)

## 5. Estatísticas Finais

In [ ]:
print("=" * 50)
print("RESUMO DA CONSOLIDAÇÃO")
print("=" * 50)
print(f"Total de registros: {len(df)}")
print(f"Registros atuais (v_current=True): {df['v_current'].sum()}")
print(f"Registros históricos: {(~df['v_current']).sum()}")
print(f"\nPor competição (source_file):")
print(df["source_file"].value_counts())
print(f"\nPor posição mapeada (todos):")
print(df["mapped_position"].value_counts())

In [ ]:
# Jogadores por posição e competição (apenas atuais)
df_current = df[df["v_current"] == True]
pivot = df_current.groupby(["source_file", "mapped_position"]).size().unstack(fill_value=0)
print("\nDistribuição por arquivo e posição (v_current=True):")
pivot

## 6. Salvar Dados Consolidados

In [ ]:
# Salvar dados consolidados (TODOS os registros, com v_current marcado)
df.to_parquet(OUTPUT_DIR / "_temp_scouts_consolidated.parquet", index=False)
print(f"Dados salvos: {OUTPUT_DIR / '_temp_scouts_consolidated.parquet'}")
print(f"Total: {len(df)} registros")
print(f"Registros atuais (v_current=True): {df['v_current'].sum()}")
print(f"\nPara filtrar apenas atuais: df[df['v_current'] == True]")